In [5]:
import tensorflow as tf
import tensorflow_io as tfio
import keras
from keras import layers
from keras.layers import Dense, Flatten, Conv2D
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
import numpy as np
import pathlib
import os
from IPython import display
import matplotlib.pyplot as plt
from glob import glob
import IPython.display as ipd

In [18]:
model = keras.models.load_model("Training/processor.h5")

OSError: Unable to open file (truncated file: eof = 26200, sblock->base_addr = 0, stored_eof = 3081946712)

In [6]:
def load_wav(filename):
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [7]:
def preprocess(file_path, label):
    wav = load_wav(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [11]:
audio_files = glob('Training/Data/WakeWord/*/*')

In [12]:
ipd.Audio(audio_files[0])

In [14]:
HB = os.path.join('Training', 'Data', 'WakeWord', 'Homebot')
EM = os.path.join('Training', 'Data', 'WakeWord', 'Empty')

pos = tf.data.Dataset.list_files(HB +'/*.wav')
neg = tf.data.Dataset.list_files(EM +'/*.wav')

homebot = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
empty = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))

ds = homebot.concatenate(empty)

In [15]:
test_audio = []
test_labels = []

for audio, label in ds:
    test_audio.append(audio.numpy())
    test_labels.append(label.numpy())
test_audio = np.array(test_audio)
test_labels = np.array(test_labels)

In [16]:
y_pred = np.argmax(md.predict(test_audio), axis=1)
y_true = test_labels

NameError: name 'md' is not defined

In [ ]:
y_pred

In [ ]:
y_true

In [ ]:
filepath, label = ds.shuffle(buffer_size=10000).as_numpy_iterator().next()
spectrogram, label = preprocess(filepath, label)
print(filepath)
plt.figure(figsize=(30,20))
plt.imshow(tf.transpose(spectrogram)[0])
plt.show

In [ ]:
ds = ds.map(preprocess)
ds = ds.cache()
ds = ds.shuffle(buffer_size=1000)
ds = ds.batch(16)
ds = ds.prefetch(8)